# 18АС | Пигасин Дмитрий | 6 Вариант

Установка Keras Tuner

In [1]:
!pip install -U keras-tuner

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=b1e0874d03acb7aa9f30619ee1a46e26334c89f9d9035490d3fadf6d9a43a8a4
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=8e2dab6422a9a4b190f7e19b2f0a97db9a1d8c638298b8879e357e27e5fe6551
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


Импорт модулей

In [20]:
import pandas as pd
import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.callbacks import Callback
from kerastuner.tuners import Hyperband
import IPython

## Подготовка данных

Загрузка данных с GDrive

In [3]:
data = pd.read_csv('/content/drive/My Drive/data_analysis/lab1/data.csv')

Просмотр датасета

In [4]:
data

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,...,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,B,C,E,A,C,T,B,G,A,A,I,E,G,J,G,BU,BC,C,AS,S,A,O,LB,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,D,C,E,E,D,T,L,F,A,A,E,E,I,K,K,BI,CQ,A,AV,BM,A,O,DP,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,B,B,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,...,A,D,D,C,E,E,A,D,L,O,A,B,E,F,H,F,A,AB,DK,A,C,AF,A,I,GK,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,D,C,E,E,D,T,I,D,A,A,E,E,I,K,K,BI,CS,C,N,AE,A,O,DJ,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,B,A,B,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,H,D,B,D,E,E,A,P,F,J,A,A,D,E,K,G,B,H,C,C,Y,BM,A,K,CK,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188313,587620,A,B,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,D,D,E,E,A,P,I,D,A,A,D,E,J,G,B,BI,EG,A,G,BM,A,L,CK,0.347403,0.785784,0.613660,0.473202,0.939556,0.242437,0.289949,0.24564,0.30859,0.32935,0.223038,0.220003,0.333292,0.208216,1198.62
188314,587624,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,D,D,E,C,A,P,K,A,A,A,E,E,H,F,B,BI,BT,A,AV,AU,E,J,DF,0.507661,0.555782,0.549770,0.802892,0.704268,0.334270,0.382000,0.63475,0.40455,0.47779,0.307628,0.301921,0.318646,0.305872,1108.34
188315,587630,A,B,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,H,D,D,D,E,C,A,P,F,C,C,A,E,F,H,F,B,BI,DM,A,W,AF,A,K,DJ,0.484469,0.785784,0.792378,0.189137,0.482436,0.345883,0.370534,0.24564,0.45808,0.47779,0.445614,0.443374,0.339244,0.503888,5762.64
188316,587632,A,B,A,A,A,A,A,A,B,B,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,C,D,C,E,C,D,T,L,J,A,A,K,G,F,J,K,BI,AI,A,AS,AJ,A,O,MD,0.438385,0.422197,0.298977,0.383428,0.340543,0.704364,0.562866,0.34987,0.44767,0.53881,0.863052,0.852865,0.654753,0.721707,1562.87


Перевод категориальных признаков в вектор (One-Hot Encoding)

In [5]:
for i in range(1,117):
  cat = pd.get_dummies(data[f'cat{i}'], prefix=f'cat{i}')
  data = pd.concat([data, cat], axis=1)
  data.drop([f'cat{i}'], axis=1, inplace=True)

Разбиение на обучающую и тестовую выборки

In [6]:
split = 0.7
random_seed = 200

data.drop(['id'], axis=1, inplace=True)

y_data = data.get('loss')
x_data = data.drop(['loss'], axis=1)
del data

x_train = x_data.sample(frac=split, random_state=random_seed)
x_test = x_data.drop(x_train.index)

y_train = y_data.sample(frac=split, random_state=random_seed)
y_test = y_data.drop(y_train.index)
del x_data
del y_data

Просмотр тренировочных данных

In [7]:
x_train

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,cat1_A,cat1_B,cat2_A,cat2_B,cat3_A,cat3_B,cat4_A,cat4_B,cat5_A,cat5_B,cat6_A,cat6_B,cat7_A,cat7_B,cat8_A,cat8_B,cat9_A,cat9_B,cat10_A,cat10_B,cat11_A,cat11_B,cat12_A,cat12_B,cat13_A,cat13_B,...,cat116_LU,cat116_LV,cat116_LW,cat116_LX,cat116_LY,cat116_M,cat116_MA,cat116_MB,cat116_MC,cat116_MD,cat116_ME,cat116_MF,cat116_MG,cat116_MH,cat116_MI,cat116_MJ,cat116_MK,cat116_ML,cat116_MM,cat116_MN,cat116_MO,cat116_MP,cat116_MQ,cat116_MR,cat116_MS,cat116_MT,cat116_MU,cat116_MV,cat116_MW,cat116_O,cat116_P,cat116_Q,cat116_R,cat116_S,cat116_T,cat116_U,cat116_V,cat116_W,cat116_X,cat116_Y
35320,0.893232,0.159990,0.280933,0.713852,0.281143,0.665834,0.377559,0.75964,0.93383,0.78770,0.711942,0.714544,0.751507,0.793129,1,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11967,0.296829,0.199504,0.634224,0.215482,0.281143,0.367466,0.562623,0.30768,0.40252,0.28677,0.392500,0.384201,0.276878,0.818243,0,1,0,1,1,0,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16312,0.637419,0.422197,0.280933,0.761209,0.696981,0.646468,0.411042,0.52450,0.64873,0.79139,0.492200,0.500382,0.689974,0.387539,1,0,0,1,1,0,1,0,0,1,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
97332,0.444104,0.737068,0.865911,0.222469,0.281143,0.471277,0.684165,0.36636,0.50420,0.34928,0.853569,0.852865,0.318646,0.356840,1,0,1,0,1,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41478,0.484469,0.681761,0.549770,0.452887,0.499798,0.350688,0.363197,0.58354,0.44352,0.39599,0.341813,0.352251,0.339244,0.714526,1,0,0,1,0,1,1,0,1,0,1,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130089,0.590735,0.488789,0.692825,0.564742,0.895958,0.437758,0.521731,0.54236,0.47900,0.51111,0.705501,0.692256,0.357400,0.602138,0,1,1,0,1,0,0,1,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
146720,0.281338,0.681761,0.745801,0.463029,0.534484,0.383459,0.396666,0.81542,0.32128,0.41743,0.607500,0.594646,0.660756,0.603388,1,0,0,1,1,0,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54392,0.372785,0.681761,0.592681,0.614134,0.874863,0.400555,0.401637,0.58934,0.35127,0.43919,0.374064,0.366307,0.611431,0.237824,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
172723,0.638758,0.488789,0.506105,0.463029,0.525831,0.618041,0.533785,0.88931,0.51890,0.60401,0.453334,0.454705,0.669664,0.825828,0,1,1,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
y_train

35320     5667.98
11967     6668.04
16312      720.40
97332     5361.27
41478      867.82
           ...   
130089     608.43
146720    2527.17
54392      641.47
172723    3532.58
22367     2974.41
Name: loss, Length: 131823, dtype: float64

## Создание нейронной сети

Загрузка предобученной модели (для оценки)

In [21]:
model = keras.models.load_model('/content/drive/My Drive/data_analysis/lab1/model.h5')

In [22]:
mae = model.evaluate(x_test, y_test, verbose=0)[1]
print(f'Средняя абсолютная ошибка = {mae}')

Средняя абсолютная ошибка = 1160.7708740234375


Описание модели

In [9]:
def build_model(hp):
  activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])
  optimizer_choice = hp.Choice('optimizer', values=['adam', 'rmsprop', 'SGD'])

  model = Sequential()
  model.add(
      Dense(
          units=hp.Int(
              'units_input',
              min_value=x_train.shape[1], 
              max_value=x_train.shape[1] * 3, 
              step=256,
          ), 
          input_dim=x_train.shape[1], 
          activation=activation_choice,
      ),
  )
  for i in range(hp.Int('num_layers', 0, 1)):
    model.add(
        Dense(
            units=hp.Int(
                'units_' + str(i),
                min_value=256,
                max_value=2048,
                step=256,
            ),
            activation=activation_choice
        )
    )
  model.add(Dense(1))

  model.compile(
      optimizer=optimizer_choice, 
      loss='mae',
      metrics=['mae'], 
  )

  return model

Создание тюнера

In [10]:
tuner = Hyperband(
    build_model,
    objective='mae', 
    max_epochs=3,
    directory='models',
    project_name='lab1',
)

In [11]:
tuner.search_space_summary()

Callback для очистки результатов в конце каждого шага обучения

In [12]:
class ClearTrainingOutput(Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)


Поиск оптимальных гиперпараметров

In [13]:
tuner.search(
    x_train,
    y_train,
    epochs=3,
    callbacks = [ClearTrainingOutput()],
)

INFO:tensorflow:Oracle triggered exit


Создание модели на основе найденных гиперпараметров

In [14]:
best_hps = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hps)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1665)              1921410   
_________________________________________________________________
dense_1 (Dense)              (None, 1280)              2132480   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1281      
Total params: 4,055,171
Trainable params: 4,055,171
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(x_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
3296/3296 - 138s - loss: 1252.9545 - mae: 1252.9545 - val_loss: 1224.4808 - val_mae: 1224.4808
Epoch 2/10
3296/3296 - 140s - loss: 1195.0238 - mae: 1195.0238 - val_loss: 1190.6909 - val_mae: 1190.6909
Epoch 3/10
3296/3296 - 140s - loss: 1176.4772 - mae: 1176.4772 - val_loss: 1179.8638 - val_mae: 1179.8638
Epoch 4/10
3296/3296 - 139s - loss: 1161.3849 - mae: 1161.3849 - val_loss: 1170.6873 - val_mae: 1170.6873
Epoch 5/10
3296/3296 - 139s - loss: 1153.0065 - mae: 1153.0065 - val_loss: 1210.7062 - val_mae: 1210.7062
Epoch 6/10
3296/3296 - 142s - loss: 1141.6442 - mae: 1141.6442 - val_loss: 1170.7046 - val_mae: 1170.7046
Epoch 7/10
3296/3296 - 144s - loss: 1132.7361 - mae: 1132.7361 - val_loss: 1182.7942 - val_mae: 1182.7942
Epoch 8/10
3296/3296 - 143s - loss: 1124.3582 - mae: 1124.3582 - val_loss: 1170.9537 - val_mae: 1170.9537
Epoch 9/10
3296/3296 - 145s - loss: 1116.3295 - mae: 1116.3295 - val_loss: 1174.0366 - val_mae: 1174.0366
Epoch 10/10
3296/3296 - 148s - loss: 1108.1803

## Оценка результатов обучения

In [16]:
mae = model.evaluate(x_test, y_test, verbose=0)[1]
print(f'Средняя абсолютная ошибка = {mae}')

Средняя абсолютная ошибка = 1160.7708740234375


Топ Kaggle   
https://www.kaggle.com/c/allstate-claims-severity/leaderboard

In [18]:
# Сохранение модели
model.save('/content/drive/My Drive/data_analysis/lab1/model.h5')